In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the pretrained embedder

In [24]:
import os
os.chdir('/Users/sebas/projects/metabolomics')
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [25]:
import torch

In [74]:
ls -lh notebooks/pretraining/results/

total 40392
-rw-r--r--  1 sebas  staff    20M Feb 28 11:41 best_model_20240228.ckpt
-rw-r--r--  1 sebas  staff    55K Feb 28 10:03 loss.png


In [68]:
!cp ./notebooks/pretraining/results/best_model-v2.ckpt  ./notebooks/pretraining/results/best_model_20240228.ckpt

In [27]:
model_path= './notebooks/pretraining/results/best_model-v2.ckpt'

In [28]:
from src.transformers.embedder import Embedder
from src.pretraining_maldi.embedder_maldi import EmbedderMaldi

In [29]:
## STEPS:
# Create an embedder object
# load the weights of embedder maldi and apply it

In [30]:
embedder= Embedder(d_model=128, n_layers=5)

In [31]:
embeddermaldi = EmbedderMaldi(d_model=128, n_layers=5)

## WEIGHTS BEFORE LOADING

In [32]:
# Inspect the weights of each layer
for name, param in embedder.named_parameters():
    print(f"Layer: {name}")
    print(param)
    print("Shape:", param.shape)

Layer: linear.weight
Parameter containing:
tensor([[ 0.0348,  0.0792,  0.0508,  ..., -0.0872, -0.0486, -0.0079],
        [ 0.0075, -0.0831, -0.0427,  ..., -0.0420, -0.0041,  0.0642],
        [ 0.0633,  0.0177, -0.0522,  ..., -0.0362,  0.0387,  0.0448],
        ...,
        [ 0.0587, -0.0779, -0.0651,  ...,  0.0326,  0.0486,  0.0530],
        [-0.0789, -0.0471, -0.0405,  ...,  0.0360,  0.0211,  0.0729],
        [-0.0175,  0.0526,  0.0873,  ...,  0.0240, -0.0376, -0.0673]],
       requires_grad=True)
Shape: torch.Size([128, 128])
Layer: linear.bias
Parameter containing:
tensor([ 0.0081,  0.0879, -0.0671,  0.0069, -0.0541,  0.0201,  0.0141,  0.0453,
        -0.0134, -0.0508,  0.0022, -0.0318,  0.0472,  0.0183,  0.0127, -0.0615,
        -0.0345,  0.0046, -0.0698,  0.0547,  0.0827, -0.0119,  0.0819, -0.0493,
        -0.0489,  0.0146, -0.0159,  0.0716, -0.0858,  0.0693,  0.0217,  0.0414,
        -0.0336,  0.0731,  0.0060,  0.0534, -0.0783, -0.0456, -0.0059, -0.0858,
        -0.0477,  0.0422,

In [67]:
# Load weights from the checkpoint
checkpoint = torch.load(model_path, map_location='cpu',)

In [34]:
# Load weights into model B from the checkpoint
checkpoint_keys = checkpoint['state_dict'].keys()
original_embedder_keys = embedder.state_dict().keys()  # Assuming `model` is your target model

# Load weights for shared layers
for key in checkpoint_keys:
    if key in original_embedder_keys:
        embedder.state_dict()[key].copy_(checkpoint['state_dict'][key])

In [35]:
embedder

Embedder(
  (linear): Linear(in_features=128, out_features=128, bias=True)
  (linear_regression): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (spectrum_encoder): SpectrumTransformerEncoderCustom(
    (peak_encoder): PeakEncoder(
      (mz_encoder): FloatEncoder()
      (int_encoder): FloatEncoder()
      (combiner): Linear(in_features=256, out_features=128, bias=False)
    )
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-4): 5 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=1024, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05

## WEIGHTS AFTER LOADING

In [37]:
# Inspect the weights of each layer
for name, param in embedder.named_parameters():
    print(f"Layer: {name}")
    print(param)
    print("Shape:", param.shape)

Layer: linear.weight
Parameter containing:
tensor([[ 0.0599,  0.0744, -0.0555,  ..., -0.0895,  0.0055, -0.0773],
        [ 0.0246,  0.1298,  0.0502,  ...,  0.0109, -0.0761, -0.1921],
        [ 0.0117, -0.1196,  0.0415,  ..., -0.0858, -0.0042,  0.1628],
        ...,
        [ 0.0520,  0.2312,  0.1127,  ..., -0.0918, -0.0143, -0.0312],
        [-0.3357,  0.1913,  0.0475,  ..., -0.2784,  0.0550, -0.0400],
        [-0.1059,  0.0350,  0.0438,  ..., -0.0439, -0.0775,  0.1274]],
       requires_grad=True)
Shape: torch.Size([128, 128])
Layer: linear.bias
Parameter containing:
tensor([ 0.0872,  0.1037, -0.0084, -0.0851,  0.1505,  0.0758, -0.0403,  0.0591,
         0.0092, -0.0625, -0.0322,  0.0330,  0.0707, -0.0603,  0.0812,  0.0742,
         0.1225,  0.0413, -0.0102, -0.0106,  0.1113,  0.0413,  0.0097,  0.1081,
        -0.0926,  0.0291, -0.0240,  0.0218, -0.0160, -0.0244,  0.1104,  0.0626,
        -0.0232,  0.0807, -0.0483, -0.0075,  0.0250, -0.0037,  0.0808, -0.0492,
        -0.0462,  0.0820,

In [38]:
embedder.named_parameters()['spectrum_encoder.transformer_encoder.layers.4.norm2.bias']

TypeError: 'generator' object is not subscriptable

In [52]:
import numpy as np

In [57]:
original_weights={}
for name, param in embedder.named_parameters():
            original_weights[name]= np.array(param.data)

In [58]:
new_weights={}
for name, param in embedder.named_parameters():
            new_weights[name]= np.array(param.data)

In [59]:
original_weights['spectrum_encoder.transformer_encoder.layers.4.norm2.bias']

array([-0.07001466,  0.11380606, -0.05894773,  0.01259844,  0.08493484,
       -0.03129856,  0.01486694, -0.05215719, -0.0055568 , -0.14563724,
       -0.11055389,  0.0227961 , -0.01865264,  0.00405181,  0.08112803,
       -0.01544514, -0.00339687,  0.02296241, -0.02957952,  0.00218931,
        0.00666927, -0.02964659,  0.06197131,  0.04962029,  0.0509708 ,
       -0.02964767, -0.02305683,  0.02698306,  0.09118502, -0.06642473,
       -0.04635222, -0.04058108,  0.04392748,  0.15005527,  0.05324572,
       -0.04787251,  0.01292375,  0.01862132,  0.02442603, -0.00781704,
        0.01287711,  0.01389454, -0.00364731,  0.08144256, -0.01420997,
        0.07320419,  0.04855328, -0.01456378, -0.06130183,  0.04054356,
        0.04400551,  0.05812364, -0.02387618, -0.1217398 , -0.00044372,
        0.1648957 , -0.01072588, -0.02684133,  0.00768973, -0.0599969 ,
       -0.00247278, -0.03385905, -0.01059776,  0.01269458,  0.02394608,
        0.00184111, -0.01503654,  0.11588462, -0.02537071,  0.01

In [63]:
np.array_equal(original_weights['spectrum_encoder.transformer_encoder.layers.0.norm2.bias'], new_weights['spectrum_encoder.transformer_encoder.layers.4.norm2.bias'])

False

In [121]:
import random
def add_noise_to_precursor_mass(sample, max_noise=0.10):
        
        added_noise_factor_0= random.uniform(-max_noise, max_noise)
        added_noise_factor_1= random.uniform(-max_noise, max_noise)
        sample["precursor_mass_0"] = sample["precursor_mass_0"] + added_noise_factor_0*(sample["precursor_mass_0"])
        sample["precursor_mass_1"] = sample["precursor_mass_1"] + added_noise_factor_1*(sample["precursor_mass_1"])
        return sample
    

In [122]:
sample={}
sample["precursor_mass_0"]=600
sample["precursor_mass_1"]=600

In [123]:
add_noise_to_precursor_mass(sample, max_noise=0.10)

{'precursor_mass_0': 639.5285597103593, 'precursor_mass_1': 614.2223702837096}

In [127]:
import torch
import torch.nn as nn

# Example input vectors (not normalized)
input1 = torch.tensor([1.0, 2.0, 3.0])
input1 = torch.tensor([-1.0, -2.0, -3.0])
# Normalize input vectors
#input1_normalized = nn.functional.normalize(input1.unsqueeze(0), p=2, dim=1)
#input2_normalized = nn.functional.normalize(input2.unsqueeze(0), p=2, dim=1)

# Compute cosine similarity
cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)
similarity = cos_sim(input1_normalized, input2_normalized)

print(similarity.item())

Seed set to 42


-0.9746319055557251


In [130]:
ls -lh notebooks/pretraining/results/

total 161224
-rw-r--r--  1 sebas  staff    20M Feb 28 14:34 best_model-v1.ckpt
-rw-r--r--  1 sebas  staff    20M Feb 28 16:25 best_model-v2.ckpt
-rw-r--r--  1 sebas  staff    20M Feb 28 11:51 best_model.ckpt
-rw-r--r--  1 sebas  staff    20M Feb 28 11:41 best_model_20240228.ckpt
-rw-r--r--  1 sebas  staff    51K Feb 28 17:10 loss.png


python(31947) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
